# Module 4: Model Context Protocol (MCP) Integration

## Connecting LLM Agents to the World

**Updated for LangChain v1 and LangGraph v1 (December 2024)**

---

### Learning Objectives

By the end of this module, you will:
1. Understand **what MCP is** and why it matters
2. Know the **architecture** of MCP (servers, clients, transports)
3. **Build an MCP server** from scratch using FastMCP
4. **Consume MCP tools** from a LangChain v1 agent
5. Understand **real-world use cases** for MCP

---

### The Problem MCP Solves

> "Every AI company is building tool integrations from scratch. It's like the web before HTTP."

#### Before MCP: The N × M Problem

```
┌─────────────┐     ┌─────────────┐     ┌─────────────┐
│   Claude    │────▶│  Custom     │────▶│   Slack     │
│             │     │  Connector  │     │             │
└─────────────┘     └─────────────┘     └─────────────┘

┌─────────────┐     ┌─────────────┐     ┌─────────────┐
│   GPT-4     │────▶│  Different  │────▶│   Slack     │
│             │     │  Connector  │     │             │
└─────────────┘     └─────────────┘     └─────────────┘
```

Every LLM × Every Tool = Explosion of custom integrations

#### After MCP: Standardized Protocol

```
┌─────────────┐                        ┌─────────────┐
│   Claude    │◀───┐                ┌──▶│   Slack     │
└─────────────┘    │    ┌─────┐    │   └─────────────┘
                   │    │     │    │
┌─────────────┐    ├────│ MCP │────┤   ┌─────────────┐
│   GPT-4     │◀───┤    │     │    ├──▶│   GitHub    │
└─────────────┘    │    └─────┘    │   └─────────────┘
                   │               │
┌─────────────┐    │               │   ┌─────────────┐
│   Llama     │◀───┘               └──▶│   Database  │
└─────────────┘                        └─────────────┘
```

One protocol to rule them all!

In [3]:
# Setup
import sys
sys.path.append('..')
from setup_llm import verify_setup, get_chat_model

verify_setup()
print("✅ Environment ready for MCP exploration")

🔍 Checking LLM Configuration...
📡 Provider: DIAL (Azure OpenAI via EPAM AI Proxy)
✅ DIAL_API_KEY is set

📋 Configuration:
   AZURE_OPENAI_ENDPOINT: https://ai-proxy.lab.epam.com
   AZURE_OPENAI_API_VERSION: 2024-08-01-preview
   AZURE_OPENAI_DEPLOYMENT_NAME: gpt-4

✅ DIAL setup verified successfully!
✅ Environment ready for MCP exploration


## Part 1: MCP Fundamentals

### What is MCP?

**Model Context Protocol (MCP)** is an open protocol developed by Anthropic that standardizes how AI applications connect to external data sources and tools.

Think of it like:
- **HTTP** for web communication
- **SQL** for database queries  
- **MCP** for AI ↔ Tool communication

### Core Components

| Component | Role | Example |
|-----------|------|---------|
| **MCP Server** | Exposes tools/resources | A server that provides GitHub tools |
| **MCP Client** | Connects to servers | Claude Desktop, your LangGraph agent |
| **Transport** | How they communicate | HTTP (network), stdio (subprocess) |
| **Tools** | Actions the LLM can take | `create_issue()`, `search_code()` |
| **Resources** | Data the LLM can read | File contents, database records |
| **Prompts** | Reusable prompt templates | Standard operating procedures |

### Transport Types

| Transport | How it Works | Best For |
|-----------|--------------|----------|
| **HTTP** (streamable-http) | Client connects to server via HTTP URL | **Production**, remote servers, multiple clients |
| **stdio** | Client spawns server as subprocess | Local debugging, CLI tools |

```
HTTP Transport (Production)
============================
┌──────────────┐    HTTP     ┌──────────────┐
│  MCP Client  │ ──────────▶ │  MCP Server  │
│  (Terminal 2)│             │  (Terminal 1)│
│              │ ◀────────── │              │
│              │  JSON/SSE   │ :8000/mcp    │
└──────────────┘             └──────────────┘

• Server runs independently (python server.py)
• Client connects via URL (http://localhost:8000/mcp)
• Multiple clients can connect simultaneously
• Easy to deploy and scale

stdio Transport (Local)
========================
┌──────────────┐   spawn    ┌──────────────┐
│  MCP Client  │ ─────────▶ │  MCP Server  │
│              │ ◀────────▶ │  (subprocess)│
│              │  stdin/out │              │
└──────────────┘            └──────────────┘

• Client spawns server as a child process
• Communication via stdin/stdout pipes
• Server lifecycle tied to client
• Good for local CLI integrations
```

**Our project uses HTTP transport** because it better demonstrates real-world architecture where servers are deployed independently.

## Part 2: Building an MCP Server 🔧

Let's build a DevOps MCP server that provides tools for:
- Checking service health
- Fetching logs  
- Running diagnostics
- Executing safe commands

### Step 1: Project Setup

First, install the required packages:

```bash
pip install mcp langchain-mcp-adapters
```

### Step 2: Server Implementation

Here's how to create an MCP server using FastMCP:

In [ ]:
# Let's examine the MCP server we've built
# (This is server.py from projects/mcp-devops-tools/)

server_code = '''
"""DevOps MCP Server - Exposes tools for service monitoring.

Run with:
    python server.py          # HTTP mode (production-like) at :8000/mcp
    python server.py --stdio  # stdio mode (local subprocess)
"""

from mcp.server.fastmcp import FastMCP
import json
import random
from datetime import datetime

# Server configuration
MCP_HOST = "localhost"
MCP_PORT = 8000

# Create the MCP server
mcp = FastMCP("devops-tools")

# ============================================================
# Tool 1: Get Service Health
# ============================================================
@mcp.tool()
async def get_service_health(service_name: str) -> dict:
    """
    Get the health status of a service.
    
    Args:
        service_name: Name of the service (e.g., 'api-gateway', 'auth-service')
    
    Returns:
        Health status including uptime, latency, and error rate
    """
    # Simulated health data (in production, call real monitoring APIs)
    statuses = ["healthy", "healthy", "healthy", "degraded", "unhealthy"]
    
    return {
        "service": service_name,
        "status": random.choice(statuses),
        "uptime": f"{random.uniform(99.0, 99.99):.2f}%",
        "latency_ms": random.randint(10, 200),
        "error_rate": f"{random.uniform(0, 2):.2f}%",
        "last_checked": datetime.now().isoformat()
    }


# ============================================================
# Tool 2: Get Logs
# ============================================================
@mcp.tool()
async def get_logs(
    service_name: str, 
    lines: int = 50, 
    level: str = "all"
) -> dict:
    """
    Fetch recent logs for a service.
    
    Args:
        service_name: Name of the service
        lines: Number of log lines to fetch (default: 50)
        level: Filter by log level: 'all', 'error', 'warn', 'info'
    """
    levels = ["INFO", "WARN", "ERROR", "DEBUG"]
    messages = [
        "Request processed successfully",
        "Cache miss, fetching from database",
        "Connection timeout, retrying...",
        "Rate limit exceeded for client",
        "Health check passed",
        "Memory usage above threshold",
    ]
    
    logs = []
    for i in range(min(lines, 20)):
        log_level = random.choice(levels)
        if level != "all" and log_level.lower() != level.lower():
            continue
        logs.append({
            "timestamp": datetime.now().isoformat(),
            "level": log_level,
            "message": random.choice(messages),
            "service": service_name
        })
    
    return {"service": service_name, "logs": logs, "total": len(logs)}


# ============================================================
# Tool 3: Run Diagnostics
# ============================================================
@mcp.tool()
async def run_diagnostics(service_name: str) -> dict:
    """
    Run diagnostic checks on a service.
    
    Args:
        service_name: Name of the service to diagnose
    """
    checks = {
        "database_connection": random.choice([True, True, True, False]),
        "cache_available": random.choice([True, True, False]),
        "dependencies_healthy": random.choice([True, True, True, False]),
        "disk_space_ok": True,
        "memory_ok": random.choice([True, True, False]),
        "cpu_ok": True,
    }
    
    failed = [k for k, v in checks.items() if not v]
    
    return {
        "service": service_name,
        "status": "pass" if not failed else "fail",
        "checks": checks,
        "failed_checks": failed,
        "recommendations": [
            f"Investigate {check}" for check in failed
        ] if failed else ["All checks passed!"]
    }


# ============================================================
# Main Entry Point - HTTP Transport (default)
# ============================================================

# Create Starlette app for HTTP transport
app = mcp.streamable_http_app()

if __name__ == "__main__":
    import argparse
    import uvicorn
    
    parser = argparse.ArgumentParser()
    parser.add_argument("--stdio", action="store_true")
    parser.add_argument("--host", default=MCP_HOST)
    parser.add_argument("--port", type=int, default=MCP_PORT)
    args = parser.parse_args()
    
    if args.stdio:
        mcp.run(transport="stdio")  # For subprocess spawning
    else:
        # HTTP transport - production-like setup using uvicorn
        print(f"🚀 Starting server at http://{args.host}:{args.port}/mcp")
        uvicorn.run(app, host=args.host, port=args.port)
'''

print("📄 MCP Server Structure:")
print("=" * 50)
print(server_code)

📄 MCP Server Structure:

"""DevOps MCP Server - Exposes tools for service monitoring."""

from mcp.server.fastmcp import FastMCP
import json
import random
from datetime import datetime

# Create the MCP server
mcp = FastMCP("devops-tools")

# ============================================================
# Tool 1: Get Service Health
# ============================================================
@mcp.tool()
async def get_service_health(service_name: str) -> dict:
    """
    Get the health status of a service.

    Args:
        service_name: Name of the service (e.g., 'api-gateway', 'auth-service')

    Returns:
        Health status including uptime, latency, and error rate
    """
    # Simulated health data (in production, call real monitoring APIs)
    statuses = ["healthy", "healthy", "healthy", "degraded", "unhealthy"]

    return {
        "service": service_name,
        "status": random.choice(statuses),
        "uptime": f"{random.uniform(99.0, 99.99):.2f}%",
        "latency_m

### Key Concepts in the Server Code

1. **`FastMCP("devops-tools")`** - Creates a named MCP server
2. **`@mcp.tool()`** - Decorator that exposes a function as an MCP tool
3. **Type hints** - MCP uses these for parameter/return schemas
4. **Docstrings** - Become the tool description shown to the LLM
5. **`mcp.streamable_http_app()`** - Returns a Starlette app for HTTP transport (run with uvicorn)

### Transport Options

| Transport | Command | URL |
|-----------|---------|-----|
| **HTTP** (default) | `python server.py` | `http://localhost:8000/mcp` |
| **stdio** | `python server.py --stdio` | N/A (subprocess) |

### Tool Design Best Practices

| Practice | Why It Matters |
|----------|---------------|
| **Clear names** | LLM selects tools by name |
| **Detailed docstrings** | LLM reads this to understand when to use |
| **Type annotations** | Enables parameter validation |
| **Structured returns** | JSON makes parsing reliable |
| **Error handling** | Graceful failures help debugging |

## Part 3: Connecting to MCP from Python 🔌

### Direct MCP Client Usage (HTTP Transport)

Before integrating with LangGraph, let's understand how to connect to an MCP server directly.

**Two-Terminal Workflow:**
1. **Terminal 1**: `python server.py` (starts HTTP server at :8000)
2. **Terminal 2**: `python simple_client.py` (connects via HTTP)

In [ ]:
# Direct MCP Client Usage over HTTP (from simple_client.py)

client_code = '''
"""Basic MCP Client - Connect via HTTP and call tools directly.

Prerequisites:
    1. Start the server: python server.py
    2. Then run this:   python simple_client.py
"""

import asyncio
import json
from mcp import ClientSession
from mcp.client.streamable_http import streamablehttp_client

# Server URL - must match server.py
MCP_SERVER_URL = "http://localhost:8000/mcp"

async def main():
    print(f"Connecting to {MCP_SERVER_URL}...")
    
    # Connect using HTTP transport
    async with streamablehttp_client(MCP_SERVER_URL) as (read, write, _):
        async with ClientSession(read, write) as session:
            # Initialize the MCP connection
            await session.initialize()
            
            # List available tools
            tools_result = await session.list_tools()
            print(f"Available tools: {len(tools_result.tools)}")
            for tool in tools_result.tools:
                print(f"  - {tool.name}: {tool.description[:50]}...")
            
            # Call a tool
            result = await session.call_tool(
                "get_service_health",
                arguments={"service_name": "api-gateway"}
            )
            
            # Parse the result
            content = result.content[0].text
            data = json.loads(content)
            print(f"\\nHealth check result:")
            print(json.dumps(data, indent=2))

if __name__ == "__main__":
    asyncio.run(main())
'''

print("📄 Simple MCP Client (HTTP Transport):")
print("=" * 50)
print(client_code)

📄 Simple MCP Client:

"""Basic MCP Client - Connect and call tools directly."""

import asyncio
import json
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client

async def main():
    # Configure connection to server
    server_params = StdioServerParameters(
        command="python",
        args=["server.py"],
    )

    # Connect using stdio transport
    async with stdio_client(server_params) as (read, write):
        async with ClientSession(read, write) as session:
            # Initialize the MCP connection
            await session.initialize()

            # List available tools
            tools_result = await session.list_tools()
            print(f"Available tools: {len(tools_result.tools)}")
            for tool in tools_result.tools:
                print(f"  - {tool.name}: {tool.description[:50]}...")

            # Call a tool
            result = await session.call_tool(
                "get_service_health",
                arg

### Connection Flow (HTTP Transport)

```
┌─────────────────────────────────────────────────────────────┐
│                  MCP HTTP Client Flow                       │
├─────────────────────────────────────────────────────────────┤
│                                                             │
│  Terminal 1: python server.py                               │
│              🚀 Server running at http://localhost:8000/mcp │
│                                                             │
│  Terminal 2: python simple_client.py                        │
│                                                             │
│  1. streamablehttp_client(url)  ──▶  Connect via HTTP       │
│                                                             │
│  2. ClientSession(read, write)  ──▶  Create session         │
│                                                             │
│  3. session.initialize()  ──▶  Handshake with server        │
│                                                             │
│  4. session.list_tools()  ──▶  Discover available tools     │
│                                                             │
│  5. session.call_tool(name, args)  ──▶  Execute tool        │
│                                                             │
└─────────────────────────────────────────────────────────────┘
```

**Key difference from stdio**: Server runs independently - you start it once, and multiple clients can connect to it.

## Part 4: LangChain v1 + MCP Integration 🚀

This is where the magic happens! We'll use `langchain-mcp-adapters` to:
1. Connect to our MCP server over HTTP
2. Convert MCP tools to LangChain tools
3. Use them with LangChain v1's `create_agent`

### The Bridge: langchain-mcp-adapters

```
┌─────────────────┐    ┌─────────────────────────┐    ┌─────────────┐
│   LangChain v1  │◀──▶│ langchain-mcp-adapters  │◀──▶│ MCP Server  │
│   create_agent  │    │                         │    │ (HTTP)      │
│                 │    │ • Converts MCP→LangChain│    │             │
│                 │    │ • Manages connections   │    │ :8000/mcp   │
│                 │    │ • HTTP/stdio transport  │    │             │
└─────────────────┘    └─────────────────────────┘    └─────────────┘
```

### What's New in LangChain v1

| Old API (Deprecated) | New API (LangChain v1) |
|---------------------|------------------------|
| `from langgraph.prebuilt import create_react_agent` | `from langchain.agents import create_agent` |
| `create_react_agent(llm, tools, state_modifier=...)` | `create_agent(model=llm, tools=tools, system_prompt=...)` |
| stdio spawn subprocess | HTTP connect to running server |

In [ ]:
# LangChain v1 + MCP Integration Code (HTTP Transport)
# Updated for LangChain v1 and LangGraph v1 (December 2024)

langgraph_code = '''
"""LangChain v1 Consumer for MCP DevOps Tools (HTTP Transport).

Prerequisites:
    1. Start the server: python server.py
    2. Then run this:   python langgraph_consumer.py
"""

import asyncio
from langchain.agents import create_agent  # LangChain v1 API
from langchain_mcp_adapters.client import MultiServerMCPClient

# Server URL - must match server.py
MCP_SERVER_URL = "http://localhost:8000/mcp"

async def main():
    # Configure MCP server connection using HTTP transport
    # Server must be running: python server.py
    mcp_config = {
        "devops-tools": {
            "transport": "streamable_http",
            "url": MCP_SERVER_URL,
            # Optional: "headers": {"Authorization": "Bearer <token>"}
        }
    }
    
    print(f"Connecting to MCP server at {MCP_SERVER_URL}...")
    
    # Connect to MCP server
    mcp_client = MultiServerMCPClient(mcp_config)
    
    # Get tools from MCP server (converted to LangChain format)
    tools = await mcp_client.get_tools()
    
    print(f"Loaded {len(tools)} tools from MCP server:")
    for tool in tools:
        print(f"  • {tool.name}")
    
    # Create LLM
    from langchain_openai import ChatOpenAI
    llm = ChatOpenAI(model="gpt-4o", temperature=0)
    
    # LangChain v1: create_agent replaces langgraph.prebuilt.create_react_agent
    agent = create_agent(
        model=llm,
        tools=tools,
        system_prompt="""You are a DevOps assistant. 
        Use the available tools to investigate issues and 
        provide actionable recommendations."""
    )
    
    # Run an investigation
    result = await agent.ainvoke({
        "messages": [("user", 
            "Check the health of the api-gateway service. "
            "If there are issues, run diagnostics and get logs."
        )]
    })
    
    # Print the agent's response
    for msg in result["messages"]:
        if hasattr(msg, "content") and msg.content:
            print(f"\\n{msg.type}: {msg.content[:500]}")

if __name__ == "__main__":
    asyncio.run(main())
'''

print("📄 LangChain v1 + MCP Integration (HTTP Transport):")
print("=" * 50)
print(langgraph_code)

📄 LangGraph + MCP Integration:

"""LangGraph Consumer for MCP DevOps Tools."""

import asyncio
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent
from langchain_mcp_adapters.client import MultiServerMCPClient

async def main():
    # Configure MCP server connection
    # The client will spawn the server as a subprocess
    mcp_config = {
        "devops-tools": {
            "command": "python",
            "args": ["server.py"],
            "transport": "stdio",
        }
    }

    # Connect to MCP server
    async with MultiServerMCPClient(mcp_config) as mcp_client:
        # Get tools from MCP server (converted to LangChain format)
        tools = mcp_client.get_tools()

        print(f"Loaded {len(tools)} tools from MCP server:")
        for tool in tools:
            print(f"  • {tool.name}")

        # Create LLM
        llm = ChatOpenAI(model="gpt-4o", temperature=0)

        # Create ReAct agent with MCP tools
        agent = create_react

### Key Points in the Integration

| Component | Purpose |
|-----------|---------|
| `MultiServerMCPClient` | Connects to one or more MCP servers |
| `mcp_config` | Defines transport type, URL, and optional headers |
| `client.get_tools()` | Returns LangChain-compatible tools |
| `create_agent` | LangChain v1 API to create agent with tools |

### Running the Demo

**Two-Terminal Workflow:**

```bash
cd projects/mcp-devops-tools

# Terminal 1: Start the MCP server
python server.py
# 🚀 Server running at http://localhost:8000/mcp

# Terminal 2: Run the client or agent
pip install -r requirements.txt

# Simple client (no LLM needed)
python simple_client.py

# LangChain agent (needs API key)
export OPENAI_API_KEY='your-key'
python langgraph_consumer.py
```

### Or use Make commands:

```bash
make server   # Terminal 1
make client   # Terminal 2 (or make agent)
```

## Part 5: MCP Ecosystem & Real-World Servers 🌍

### Popular MCP Servers

The MCP ecosystem is growing rapidly. Here are some useful servers:

| Server | Description | Use Case |
|--------|-------------|----------|
| **@modelcontextprotocol/server-filesystem** | File system access | Read/write files |
| **@modelcontextprotocol/server-github** | GitHub API | Issues, PRs, code search |
| **@modelcontextprotocol/server-postgres** | PostgreSQL queries | Database operations |
| **@modelcontextprotocol/server-slack** | Slack integration | Messages, channels |
| **@modelcontextprotocol/server-puppeteer** | Browser automation | Web scraping, testing |

### Using Pre-built Servers

You can connect to multiple MCP servers simultaneously:

```python
mcp_config = {
    # Your custom server
    "devops-tools": {
        "command": "python",
        "args": ["server.py"],
        "transport": "stdio",
    },
    # Official GitHub server
    "github": {
        "command": "npx",
        "args": ["-y", "@modelcontextprotocol/server-github"],
        "transport": "stdio",
        "env": {
            "GITHUB_TOKEN": os.getenv("GITHUB_TOKEN")
        }
    },
    # File system server
    "filesystem": {
        "command": "npx",
        "args": [
            "-y", 
            "@modelcontextprotocol/server-filesystem",
            "/path/to/allowed/directory"
        ],
        "transport": "stdio",
    }
}
```

### Claude Desktop Integration

MCP servers can also be used with Claude Desktop. Add to `claude_desktop_config.json`:

```json
{
  "mcpServers": {
    "devops-tools": {
      "command": "python",
      "args": ["/full/path/to/server.py"]
    }
  }
}
```

## Part 6: Beyond Tools - Resources and Prompts 📚

MCP isn't just about tools. It also supports:

### Resources

Resources let the LLM read data (like files, database records):

```python
@mcp.resource("config://services")
async def get_services_config() -> str:
    """Returns the list of monitored services."""
    return json.dumps({
        "services": ["api-gateway", "auth-service", "database"],
        "environment": "production"
    })
```

### Prompts

Prompts are reusable templates the LLM can use:

```python
@mcp.prompt("incident_template")
async def incident_template(severity: str, service: str) -> str:
    """Standard incident report template."""
    return f"""
    # Incident Report
    
    **Severity**: {severity}
    **Service**: {service}
    **Time**: {{current_time}}
    
    ## Summary
    {{summary}}
    
    ## Root Cause
    {{root_cause}}
    
    ## Resolution
    {{resolution}}
    """
```

### Complete MCP Server Structure

```
┌────────────────────────────────────────┐
│           MCP Server                    │
├────────────────────────────────────────┤
│                                        │
│  Tools (@mcp.tool)                     │
│  ├── get_service_health()              │
│  ├── get_logs()                        │
│  └── run_diagnostics()                 │
│                                        │
│  Resources (@mcp.resource)             │
│  ├── config://services                 │
│  └── docs://runbooks                   │
│                                        │
│  Prompts (@mcp.prompt)                 │
│  ├── incident_template                 │
│  └── escalation_guide                  │
│                                        │
└────────────────────────────────────────┘
```

## Part 7: MCP Best Practices ✅

### Security Considerations

| Risk | Mitigation |
|------|------------|
| **Arbitrary code execution** | Whitelist allowed commands |
| **Data exposure** | Implement authentication |
| **Resource exhaustion** | Add rate limiting |
| **Prompt injection** | Validate all inputs |

### Design Principles

1. **Single Responsibility**: Each tool does one thing well
2. **Clear Naming**: `get_service_logs` > `fetch_data`
3. **Detailed Descriptions**: Help the LLM choose correctly
4. **Graceful Errors**: Return error info, don't crash
5. **Idempotency**: Safe to call multiple times

### When to Use MCP vs Direct Integration

| Use MCP When | Use Direct Integration When |
|--------------|----------------------------|
| Building reusable tools | One-off integration |
| Multi-LLM support needed | Single LLM application |
| Team sharing tools | Personal project |
| Complex tool ecosystem | Simple tool needs |
| Production deployment | Quick prototype |

## 🎯 Hands-On Exercise

### Task: Extend the MCP DevOps Server

Add a new tool to the server that creates incident tickets:

```python
@mcp.tool()
async def create_incident(
    title: str,
    severity: str,
    service: str,
    description: str
) -> dict:
    """
    Create a new incident ticket.
    
    Args:
        title: Brief title of the incident
        severity: 'critical', 'high', 'medium', or 'low'
        service: Affected service name
        description: Detailed description
    """
    # Your implementation here
    pass
```

**Requirements**:
1. Validate severity is one of the allowed values
2. Return a ticket ID and creation timestamp
3. Include a link to the (mock) ticket

**Bonus**:
- Add a `@mcp.resource()` for listing open incidents
- Add a `@mcp.prompt()` for incident templates

## Summary

### What We Learned

1. **MCP solves the N×M problem** - One protocol for all LLM-tool connections
2. **FastMCP makes servers easy** - Just decorate your functions
3. **Two transports**: HTTP (production, remote) and stdio (local, subprocess)
4. **langchain-mcp-adapters** bridges MCP and LangChain v1
5. **MCP has 3 primitives**: Tools, Resources, and Prompts

### Key Takeaways

```
┌─────────────────────────────────────────────────────────────┐
│                     MCP Mental Model                        │
├─────────────────────────────────────────────────────────────┤
│                                                             │
│  MCP Server = Tool provider (like an API server)            │
│  MCP Client = Tool consumer (your agent)                    │
│  Transport  = How they talk (HTTP, stdio)                   │
│                                                             │
│  Tools     = Actions (functions the LLM can call)           │
│  Resources = Data (things the LLM can read)                 │
│  Prompts   = Templates (reusable prompt patterns)           │
│                                                             │
└─────────────────────────────────────────────────────────────┘
```

### Resources

- **MCP Specification**: https://spec.modelcontextprotocol.io
- **MCP GitHub**: https://github.com/modelcontextprotocol
- **Server Examples**: https://github.com/modelcontextprotocol/servers
- **langchain-mcp-adapters**: https://github.com/langchain-ai/langchain-mcp-adapters
- **Workshop Project**: `projects/mcp-devops-tools/`

---

### 🎉 Congratulations!

You've completed Session 3 of the Agentic AI Workshop!

**What you've mastered:**
- ✅ Evaluation frameworks for LLM agents (DeepEval)
- ✅ Observability and cost optimization (Langfuse)
- ✅ Security guardrails and best practices
- ✅ Common anti-patterns and how to avoid them
- ✅ MCP for tool interoperability

**Next Steps:**
- Integrate Langfuse into your incident-postmortem project
- Add DeepEval tests for quality assurance
- Build your own MCP servers for your tools
- Apply these patterns to production systems